# Data Engineer Test

This notebook will contain the code used for the test and will also play the part of the report on my work.

## Part I

### Step 1

In this step, we install the wget library as it makes it easy to download files from urls.

In [1]:
!pip install wget
import wget 

wget.download('http://base-donnees-publique.medicaments.gouv.fr/telechargement.php?fichier=CIS_bdpm.txt')

wget.download('http://base-donnees-publique.medicaments.gouv.fr/telechargement.php?fichier=CIS_COMPO_bdpm.txt')


'CIS_COMPO_bdpm.txt'

### Step 2

In [3]:
import tarfile
tf = tarfile.open('./exercice/headers.tgz')
tf.extractall('.')

### Step 3

In this step, we read the downloaded files and fill the NaN parts with empty strings as most columns are of type text. 

In [216]:
import pandas as pd 

spec = pd.read_csv('CIS_bdpm.txt', sep='\t', header=None, encoding='cp1252').fillna('')
comp = pd.read_csv('CIS_COMPO_bdpm.txt', sep='\t', header=None, encoding='cp1252').fillna('')

with open('./headers/cis_headers.txt','r') as f : 
    spec_head = [i.strip() for i in f.readlines()]

with open('./headers/compo_headers.txt','r') as f : 
    comp_head = [i.strip() for i in f.readlines()]
    
assert len(comp_head) == comp.shape[1]
assert len(spec_head) == spec.shape[1]

spec.columns = spec_head
comp.columns = comp_head

### Step 4

In this step, we have to install docker and create a Dockerfile to build an image with mysql running in it. The Dockerfile is in ./results/mysql folder.

Most of this step was done on command line and is not present on the notebook. 

In this notebook, we connect to the mysql server on the docker container and save our pandas DataFrames into it. 

In [217]:
!pip install sqlalchemy

In [1]:
import sqlalchemy as db

# specify database configurations
config = {
    'host': 'localhost',
    'port': 3306,
    'user': 'newuser',
    'password': '1234',
    'database': 'db'
}
db_user = config.get('user')
db_pwd = config.get('password')
db_host = config.get('host')
db_port = config.get('port')
db_name = config.get('database')
# specify connection string
connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'
# connect to database
engine = db.create_engine(connection_str)
connection = engine.connect()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'localhost' ([Errno 111] Connection refused)")
(Background on this error at: http://sqlalche.me/e/e3q8)

In [221]:
spec.to_sql(con=connection, name='CIS_bdpm', if_exists='replace',index=False)
comp.to_sql(con=connection, name='COMPO_bdpm', if_exists='replace', index=False)

In [222]:
# !docker build -t mysql-file .
# !docker run --name test -p3306:3306 -d mysql-file
# !docker exec -i test mysqldump -unewuser -p1234 db > bdpm.sql

### Step 5

In this step, we had to create an sql query to join both dataframes on the cis column. The sql file can be found in ./results/mysql folder.

In [224]:
with open("./results/mysql/bdpm_request.sql") as f : 
    sql_query = f.readline().strip()

In [225]:
joined = pd.read_sql(sql_query, connection)

In [226]:
joined.head(5)

,cis,longname,form,pathway,auth_status,procedure,status,market_date,unknown,ema,...,collectivity,cis,form,sub_code,sub_name,sub_dose,ref_dose,sub_nature,link,unknow
0,60002746,"ACTAEA RACEMOSA FERRIER, degré de dilution com...",granules et solution en gouttes en gouttes,orale;sublinguale,Autorisation active,Enreg homéo (Proc. Nat.),Commercialisée,03/01/2008,,,...,Non,60002746,granules,5319,ACTAEA RACEMOSA POUR PRÉPARATIONS HOMÉOPATHIQUES,2CH à 30CH et 4DH à 60DH,un comprimé,SA,7,
1,60002746,"ACTAEA RACEMOSA FERRIER, degré de dilution com...",granules et solution en gouttes en gouttes,orale;sublinguale,Autorisation active,Enreg homéo (Proc. Nat.),Commercialisée,03/01/2008,,,...,Non,60002746,solution buvable en gouttes,5319,ACTAEA RACEMOSA POUR PRÉPARATIONS HOMÉOPATHIQUES,2CH à 30CH et 4DH à 60DH,un comprimé,SA,8,
2,60007960,"AFTAGEL, gel buccal",gel,voie buccale autre,Autorisation active,Procédure nationale,Commercialisée,06/08/1996,,,...,Non,60007960,gel,931,SULFATE DE ZINC,"0,5 g",100 ml,SA,2,
3,60007960,"AFTAGEL, gel buccal",gel,voie buccale autre,Autorisation active,Procédure nationale,Commercialisée,06/08/1996,,,...,Non,60007960,gel,5432,CHLORHYDRATE DE LIDOCAÏNE,"0,6 g",100 ml,SA,1,
4,60025054,"ADEMPAS 1,5 mg, comprimé pelliculé",comprimé pelliculé,orale,Autorisation active,Procédure centralisée,Commercialisée,27/03/2014,,EU/1/13/907,...,Non,60025054,comprimé,82922,RIOCIGUAT,"1,50 mg",un comprimé,SA,1,


### Step 6

This step is a data cleaning step. We start by a simple filter on the joined dataset and then we want to clean the information in type text column. We used a regex pattern to replace only the "," between number in '.' to be able to remove all informations after ','. We then lowered and removed the accents.

In [69]:
filtered_joined = joined.loc[joined['sub_nature'] == "SA"]

In [112]:
import re 
!pip install unidecode
import unidecode
pattern = r'([+-]?[0-9]+),([0-9]+)'
func = lambda x: unidecode.unidecode(re.sub(pattern, r'\1.\2', x.lower() ).split(',')[0])
filtered_joined['normalized_longname'] = filtered_joined['longname'].apply(func)

/home/amine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Step 7

In this step, the goal is to redo the previous step using sql queries. the sql queries will be saved in ./results/mysql/ folder. 

The following cells will output an error here in the notebook but will run the sql query as wanted.

In [227]:
with open("./results/mysql/filtered_bdpm_request.sql") as f : 
    new_sql_query = f.readlines()
print(new_sql_query)

["SELECT * FROM CIS_bdpm AS t1 INNER JOIN COMPO_bdpm AS t2 ON t1.cis=t2.cis WHERE t2.sub_nature='SA'\n", 'ALTER TABLE joined ADD normalized_longname text\n', 'UPDATE joined SET normalized_longname=longname']


In [474]:
sql_filtered_joined = pd.read_sql(new_sql_query[0].strip(), connection)
sql_filtered_joined.to_sql(con=connection, name='joined', if_exists='replace', index=False)

In [475]:
_ = pd.read_sql('ALTER TABLE joined ADD normalized_longname text', connection)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [476]:
# pd.read_sql('UPDATE joined SET normalized_longname=LOWER(longname)', connection)

In [477]:
pd.read_sql("UPDATE joined SET normalized_longname=REGEXP_REPLACE(LOWER(longname), '([+-]?[0-9]+),([0-9]+)', '$1.$2')", connection)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [478]:
# pd.read_sql('UPDATE joined SET normalized_longname=CONVERT(normalized_longname USING latin1 )', connection)
pd.read_sql("UPDATE joined SET normalized_longname=SUBSTRING_INDEX(normalized_longname,',',1)", connection)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [479]:
a= pd.read_sql('SELECT * FROM joined ', connection)

In [480]:
print(a['normalized_longname'][0])
print(a['longname'][0])

abelmoschus boiron
ABELMOSCHUS BOIRON, degré de dilution compris entre 4CH et 30CH ou entre 7DH et 60DH


In [481]:
print(a['normalized_longname'][24227])
print(a['longname'][24227])

zavicefta 2 g/0.5 g
ZAVICEFTA 2 g/0,5 g, poudre pour solution à diluer pour perfusion


## Conclusion of Part I : 

The first part took me almost 6 to 8 hours of work. Steps 1,2,3,5 and 6 were quite easy and straight forward and took almost 30 minutes. Step 4 represents my first encounted with Docker and what it means and what it's used for. It took me a lot of time to understand the concepts and be able to create the dockerfile almost 4 to 5 hours. The last step 7 took almost 1 or two hours because i had never used regex in mysql and the syntax was not easy to find. 

## Part II 

### Step 1

In this part, we get the html code for the wikipedia page and parse the h2 et p tags in order then just rearrange them into paragraphs by dividing with the h2 ones.

In [152]:
import requests

r = requests.get("https://fr.wikipedia.org/wiki/Zopiclone")

!pip install beautifulsoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [525]:
l = soup.find_all(['h2','p'])
h2s = soup.find_all('h2')
h2s_text = [i.text.strip() for i in h2s]

In [535]:
pars = []
out = ''
for i in l : 
    if i.text.strip() in h2s_text : 
        pars.append(out)
        test = i.find_all('span',{'class':'mw-headline'})
        if len(test) == 0 : 
            out = i.text.strip() + '\n'
        else : 
            out = ' '.join([j.text.strip() for j in test]) + '\n'
    else : 
        out = out + i.text.strip() + '\n'

In [536]:
for i in pars : 
    print(i)


Urinaire et fécale
orale
La zopiclone est un hypnotique de la famille chimique des cyclopyrrolones, apparenté aux benzodiazépines[2]. Elle est commercialisée notamment sous le nom d'Imovane. La zopiclone possède des propriétés hypnotiques, sédatives, anxiolytiques, myorelaxantes et amnésiantes.
Elle fait partie des hypnotiques les plus couramment utilisés, avec le zolpidem et le zaleplon, tous les trois souvent qualifiés de Z-Drugs. Leur usage a supplanté celui des benzodiazépines en raison d'une demi-vie moins importante et d'effets secondaires réputés moins courants.
Le traitement avec la zopiclone était originellement recommandé pour une utilisation à court terme, c’est-à-dire pour deux semaines[3],[4].

Sommaire

Pharmacologie
La zopiclone est un hypnotique puissant, et se comporte de la même manière que les benzodiazépines , en tant qu'agoniste GABA, et donc comme inhibiteur du système nerveux central.

Indications
Le traitement est indiqué pour le traitement des insomnies aiguës

### Step 2

I couldn't do this step because installing the ELK stack on my computer was pretty hard and i couldn't do it. I had to install SElinux and set it to permissive however my pc is running on ubuntu 18.04 and this keep crashing it. 

## Conclusion of Part II 

This part took me almost 1 hour as i only did the first step. 